In [1]:
!nvidia-smi

Sun Jun  4 01:54:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
class CFG:
    env = "local"
    input_dir = None
    output_dir = None
    competition_name = "tlvmc-parkinsons-freezing-gait-prediction"
    target_cols = ["StartHesitation", "Turn", "Walking"]

In [3]:
from pathlib import Path

print(f"env is {CFG.env}.")
CFG.input_dir = Path("../mnt/input/")
CFG.output_dir = Path("./")

env is kaggle.


In [4]:
import gc
import os
import glob
import time
import math
import pathlib
import random
import warnings
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm
from IPython.display import display
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.metrics import accuracy_score, average_precision_score
from sklearn.preprocessing import StandardScaler, RobustScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
print(f"transformers.__version__: {transformers.__version__}")

transformers.__version__: 4.27.4


In [5]:
warnings.filterwarnings("ignore")

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Utils

In [7]:
def preprocess(numerical_array, mask_array):
    attention_mask = mask_array == 0
    return {
        "input_data_numerical_array": numerical_array,
        "input_data_mask_array": mask_array,
        "attention_mask": attention_mask,
    }

In [8]:
class FogDataset(Dataset):
    def __init__(
        self,
        numerical_array,
        mask_array,
        train=True,
        y=None,
    ):
        self.numerical_array = numerical_array
        self.mask_array = mask_array
        self.train = train
        self.y = y

    def __len__(self):
        return len(self.numerical_array)

    def __getitem__(self, item):
        data = preprocess(
            self.numerical_array[item],
            self.mask_array[item],

        )

        # Return the processed data where the lists are converted to `torch.tensor`s
        if self.train:
            return {
              "input_data_numerical_array": torch.tensor(data["input_data_numerical_array"], dtype=torch.float32),
              "input_data_mask_array": torch.tensor(data["input_data_mask_array"], dtype=torch.long),
              "attention_mask": torch.tensor(data["attention_mask"], dtype=torch.bool),
              "y": torch.tensor(self.y[item], dtype=torch.float32),
            }
        else:
            return {
              "input_data_numerical_array": torch.tensor(data["input_data_numerical_array"], dtype=torch.float32),
              "input_data_mask_array": torch.tensor(data["input_data_mask_array"], dtype=torch.long),
              "attention_mask": torch.tensor(data["attention_mask"], dtype=torch.bool),
            }

In [9]:
def make_pred(test_loader, model):
    preds = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    for d in tk0:
        input_data_numerical_array = d["input_data_numerical_array"].to(device)
        input_data_mask_array = d["input_data_mask_array"].to(device)
        attention_mask = d["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_data_numerical_array, input_data_mask_array, attention_mask)

        preds.append(outputs.sigmoid().detach().cpu().numpy())

    preds = np.concatenate(preds)
    return preds

# Data Loading

In [10]:
DEFOG_DATA_PATH = "../mnt/input/train/notype/*.csv"

# Settings

In [11]:
defog_path1 = [f"../mnt/output/pd-exp222/fold{i}_best.pth" for i in [0, 1, 2, 3, 4]]

In [12]:
class Defog1Model(nn.Module):
    def __init__(
        self,
        dropout=0.2,
        input_numerical_size=9,
        numeraical_linear_size=64,
        model_size=128,
        linear_out=128,
        out_size=3,
    ):
        super(Defog1Model, self).__init__()
        self.numerical_linear = nn.Sequential(
                nn.Linear(input_numerical_size, numeraical_linear_size),
                nn.LayerNorm(numeraical_linear_size)
            )
        self.lstm = nn.GRU(
            numeraical_linear_size,
            model_size,
            num_layers=2,
            batch_first=True,
            bidirectional=True,
        )
        self.linear_out = nn.Sequential(
            nn.Linear(model_size*2, linear_out),
            nn.LayerNorm(linear_out),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(linear_out, out_size),
        )
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    # Set forget-gate bias to 1
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)

    def forward(self, numerical_array, mask_array, attention_mask):
        numerical_embedding = self.numerical_linear(numerical_array)
        output, _ = self.lstm(numerical_embedding)
        output = self.linear_out(output)
        return output

In [13]:
defog_model_list1 = []
for path in defog_path1:
    model = Defog1Model()
    state = torch.load(path, map_location=torch.device("cpu"))
    model.load_state_dict(state["model"])
    model = model.to(device)
    model.eval()
    defog_model_list1.append(model)
    print(f"load weights from {path}")

load weights from /kaggle/input/pd-exp222/fold0_best.pth
load weights from /kaggle/input/pd-exp222/fold1_best.pth
load weights from /kaggle/input/pd-exp222/fold2_best.pth
load weights from /kaggle/input/pd-exp222/fold3_best.pth
load weights from /kaggle/input/pd-exp222/fold4_best.pth


In [14]:
df_all = []

In [15]:
# =========================
# defog1
# =========================
seq_len = 30000
shift = 15000
offset = 7500
bs = 128
w = 1.0
print(seq_len, shift, offset)
print(w)

defog_list = glob.glob(DEFOG_DATA_PATH)
for p in tqdm(defog_list):
    id_values = p.split("/")[-1].split(".")[0]
    df = pd.read_csv(p)

    # create features
    cols = ["AccV", "AccML", "AccAP"]
    for c in cols:
        df[f"{c}_lag_diff"] = df[c].diff()
        df[f"{c}_lead_diff"] = df[c].diff(-1)

    # scaling
    num_cols = [
        "AccV", "AccML", "AccAP",
        "AccV_lag_diff", "AccV_lead_diff",
        "AccML_lag_diff", "AccML_lead_diff",
        "AccAP_lag_diff", "AccAP_lead_diff",
    ]
    sc = StandardScaler()
    df[num_cols] = sc.fit_transform(df[num_cols].values)
    df[num_cols] = df[num_cols].fillna(0)

    num = df[num_cols].values
    time_values = df["Time"].values

    batch = (len(df)-1) // shift
    batch = max(1, batch)

    num_array = np.zeros([batch, seq_len, len(num_cols)])
    mask_array = np.zeros([batch, seq_len], dtype=int)
    pred_use_array = np.zeros([batch, seq_len], dtype=int)
    time_array = np.zeros([batch, seq_len], dtype=int)

    if len(df) <= seq_len:
        num_array[0, :len(num), :] = num
        mask_array[0, :len(num)] = 1
        pred_use_array[0, :len(num)] = 1
        time_array[0, :len(num)] = time_values
    else:
        for n, b in enumerate(range(batch)):
            if b == (batch - 1):
                # get values
                num_ = num[b*shift:]
                time_ = time_values[b*shift:]
                # insert values
                num_array[b, :len(num_), :] = num_
                mask_array[b, :len(num_)] = 1
                pred_use_array[b, offset:len(num_)] = 1
                time_array[b, :len(num_)] = time_
            elif b == 0:
                # get values
                num_ = num[0:seq_len]
                time_ = time_values[0:seq_len]
                # insert values
                num_array[0, :, :] = num_
                mask_array[0, :] = 1
                pred_use_array[0, :seq_len-offset] = 1
                time_array[0, :] = time_
            else:
                # get values
                num_ = num[b*shift:b*shift+seq_len]
                time_ = time_values[b*shift:b*shift+seq_len]
                # insert values
                num_array[b, :, :] = num_
                mask_array[b, :] = 1
                pred_use_array[b, offset:seq_len-offset] = 1
                time_array[b, :] = time_

    test_dataset = FogDataset(num_array, mask_array, train=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

    for n, m in enumerate(defog_model_list1):
        if n == 0:
            pred = make_pred(test_loader, m) / len(defog_model_list1)
        else:
            pred += make_pred(test_loader, m) / len(defog_model_list1)

    pred_list = []
    for i in range(batch):
        mask_ = pred_use_array[i]
        pred_ = pred[i,mask_ == 1,:]
        time_ = time_array[i, mask_ == 1]
        df_ = pd.DataFrame()
        df_["StartHesitation"] = pred_[:, 0] * w
        df_["Turn"] = pred_[:, 1] * w
        df_["Walking"] = pred_[:, 2] * w
        df_["Time"] = time_
        df_["Id"] = id_values
        pred_list.append(df_)

    pred = pd.concat(pred_list).reset_index(drop=True)
    df_all.append(pred)

30000 15000 7500
1.0


  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
df_all = pd.concat(df_all).reset_index(drop=True)
df_all = df_all.groupby(["Id", "Time"])[["StartHesitation", "Turn", "Walking"]].sum().reset_index()
print(df_all.shape)
df_all.head()

(10251114, 5)


,Id,Time,StartHesitation,Turn,Walking
0,02ab235146,0,0.000353,0.002108,0.003199
1,02ab235146,1,0.000308,0.002079,0.002969
2,02ab235146,2,0.000248,0.002094,0.002508
3,02ab235146,3,0.000220,0.002141,0.002374
4,02ab235146,4,0.000203,0.002258,0.002648


In [17]:
df_all.to_csv(CFG.input_dir / "notype_targets_from_exp222.csv", index=False)